<a href="https://colab.research.google.com/github/ikanx101/G-Colab/blob/main/AppSheet_Converter%20versi%20Lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ___APPSHEET-SALES CONVERTER___

_Converter_ ini dibuat untuk mengubah data hasil _input_ __AppSheet__ ke bentuk tabular yang lebih _excel-user-friendly_.

Cara pakai:

1. _Upload_ dua _files_ yang dibutuhkan, yakni:
    - Data __AppSheet__.
    - Data _pricelist_.
1. Isi _form_ sesuai dengan nama _files_ tersebut.
1. Klik `Runtime` >> `Run all`.
1. Tunggu hingga proses selesai (sampai muncul pesan `PROSES SELESAI`). Kira-kira akan memakan waktu 5-10 menit.
1. Akan muncul _file_ baru bernama `hasil_konversi.xlsx`. Silakan di-_download_.

---

Catatan: Karena yang kita gunakan saat ini adalah _server_ gratisnya Google, jadi kita harus selalu _install_ beberapa _packages_ setiap kali _run_ skrip ini. Konsekuensinya _runtime_-nya lebih lama.

_Last Update_: 5 Oktober 2023

In [ ]:
#@title Masukkan Nama Files

rm(list=ls())
nama_file_appsheet <- "Call & Omzet (14).xlsx" #@param {type:"string"}
nama_file_pricelist <- "Template Harga (1).xlsx" #@param {type:"string"}

install.packages("reshape2")
install.packages("janitor")
install.packages("openxlsx")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘plyr’, ‘Rcpp’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘snakecase’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
#@title Proses Perhitungan (harap tunggu hingga selesai)

# rename files
nama_file_utama = nama_file_appsheet
nama_file_harga = nama_file_pricelist

# panggil libraries
library(readxl)
library(dplyr)
library(tidyr)
library(reshape2)

# ========================
# extract data target utama
data = read_excel(nama_file_utama) %>% janitor::clean_names()

# ========================
# extract database produk
dbase = read_excel(nama_file_harga,
                   col_types = c("text", "text", "numeric")) %>%
  janitor::clean_names() %>%
  mutate(item_standar = janitor::make_clean_names(item),
         brand = ifelse(brand == "TS","Tropicana Slim",brand),
         brand = ifelse(brand == "NS","NutriSari",brand))

# ========================
# ambil informasi yang diperlukan
header_data = colnames(data)
nama_item = dbase$item_standar
item_yg_dijual = header_data[header_data %in% nama_item]

# ========================
# ========================
# kita bagi-bagi datanya berdasarkan informasi yang ada
# pertama dari produk
data_1 = data[colnames(data) %in% c("id",item_yg_dijual)]
data = data[!colnames(data) %in% item_yg_dijual]

# kedua dari gimmick
data_2 = data %>% select(id,contains("gimmick"))
data = data %>% select(-contains("gimmick"))

# ketiganya sisanya
data_3 = data %>% select(-starts_with("pg"),
                         -starts_with("sec"),
                         -contains("omzet"),
                         -transaksi_penjualan)

# ========================
# ========================
# proses pengerjaannya mungkin akan rumit. kenapa?
# kalau kita lihat di sheet after, banyaknya baris akan tergantung dari banyaknya baris yang ada di gimmick dan produk yang terjual.
# maka dari itu, lebih baik semua data dbuat dalam bentuk list saja.
# nanti tinggal ditempel saja ke kanan.
data_1 = data_1 %>% split(.,.$id)
data_2 = data_2 %>% split(.,.$id)
data_3 = data_3 %>% split(.,.$id)

# kita siapkan rumahnya dulu
ikanx = vector("list",length(data_1))

# sekarang kita mulai looping dari i = 1 sampai selesai
for(i in 1: length(data_1)){
  temp_1 = data_1[[i]] %>% as.data.frame()
  temp_2 = data_2[[i]] %>% as.data.frame()
  temp_3 = data_3[[i]] %>% as.data.frame()

  # sekarang kita akan kerjakan yang data_2
  # kita rapikan gimmick
  # rules: saat tidak ada gimmick, maka sisanya dbuat nol alias NA

  # perubahan terbaru 5 Oktober 2023
  # kita ubah dulu jika pemberian gimmicknya NA
  if(is.na(temp_2$pemberian_gimmick)){temp_2$pemberian_gimmick = "Tidak ada"}


  if(temp_2$pemberian_gimmick == "Ada"){
    temp_2 =
      temp_2 %>%
      melt(id.vars = c("id","pemberian_gimmick")) %>%
      mutate(variable = ifelse(grepl("item",variable),
                               "item_gimmick",
                               "qty_gimmick")
      )
    temp_2_1 =
      temp_2 %>%
      filter(grepl("item",variable)) %>%
      select(-variable) %>%
      rename(item_gimmick = value)
    temp_2_2 =
      temp_2 %>%
      filter(grepl("qty",variable)) %>%
      select(-variable) %>%
      rename(qty_gimmick = value) %>%
      select(qty_gimmick)
    temp_2 = cbind(temp_2_1,temp_2_2)
  } else {
    temp_2 = data.frame(
      id = temp_2$id,
      pemberian_gimmick = temp_2$pemberian_gimmick,
      item_gimmick = NA,
      qty_gimmick = NA
    )
  }

  # sekarang kita akan kerjakan yang data_1
  # item penjualan kita buat tabular
  temp_1 =
    temp_1 %>%
    melt(id.vars = "id") %>%
    filter(!is.na(value)) %>%
    rename(item_standar = variable) %>%
    merge(dbase) %>%
    mutate(omzet = value*harga) %>%
    select(-item_standar) %>%
    rename(qty_penjualan = value,
           item_penjualan = item) %>%
    relocate(id,item_penjualan,brand,qty_penjualan,harga,omzet)

  # sekarang saatnya moment of truth
  m1 = nrow(temp_1)
  m2 = nrow(temp_2)
  m3 = nrow(temp_3)

  max_m = max(c(m1,m2,m3))

  if(m1 < max_m){
    temp_1[(m1+1):max_m,] = NA
  }
  if(m2 < max_m){
    temp_2[(m2+1):max_m,] = NA
  }
  if(m3 < max_m){
    temp_3[(m3+1):max_m,] = temp_3[1,]
  }

  temp_1$id = NULL
  temp_2$id = NULL
  temp_3$id = NULL

  final = cbind(temp_3,temp_2,temp_1)
  ikanx[[i]] = final
  print(i)
}

# ========================
# saatnya kita gabung kembali
printed_data = do.call(rbind,ikanx)

openxlsx::write.xlsx(printed_data,"hasil_konversi.xlsx")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 132
[1] 133
[1] 134
[1] 135
[1] 136
[1] 137
[1] 138
[1] 

In [ ]:
#@title
print("PROSES SELESAI")
print("file hasil konversi tersimpan dalam nama: hasil_konversi.xslx")

[1] "PROSES SELESAI"
[1] "file hasil konversi tersimpan dalam nama: hasil_konversi.xslx"
